In [59]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [60]:
movies_data = pd.read_csv('u.item', sep='|', names = col_name_movies, encoding='latin-1')
col_name_rating = ['user_id', 'item_id', 'rating', 'timestamp']
rating = pd.read_csv('u.data', sep='\t', names = col_name_rating)

col_name_user = ['user_id', 'age', 'gender', 'occupation', 'zip_code']
users = pd.read_csv('u.user', sep='|', names = col_name_user)

col_name_movies = ['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 
                   'Adventure','Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 
                   'Film-Noir', 'Horror','Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']


## KNN Algorithm from scratch

In [61]:
n_users = rating.user_id.unique().shape[0]
n_items = rating.item_id.unique().shape[0]

In [62]:
user_data_matrix = np.zeros((n_users, n_items))
for line in rating.itertuples():
    user_data_matrix[line[1]-1, line[2]-1] = line[3]

In [63]:
mean_user_rating = np.nanmean(user_data_matrix,axis = 1)

In [64]:
ratings_diff = (user_data_matrix - mean_user_rating[:, np.newaxis]) 
rating_diff_new = np.nan_to_num(ratings_diff)

In [65]:
rating_diff_new

array([[ 4.41617122,  2.41617122,  3.41617122, ..., -0.58382878,
        -0.58382878, -0.58382878],
       [ 3.86325803, -0.13674197, -0.13674197, ..., -0.13674197,
        -0.13674197, -0.13674197],
       [-0.08977408, -0.08977408, -0.08977408, ..., -0.08977408,
        -0.08977408, -0.08977408],
       ..., 
       [ 4.9470868 , -0.0529132 , -0.0529132 , ..., -0.0529132 ,
        -0.0529132 , -0.0529132 ],
       [-0.20035672, -0.20035672, -0.20035672, ..., -0.20035672,
        -0.20035672, -0.20035672],
       [-0.34066587,  4.65933413, -0.34066587, ..., -0.34066587,
        -0.34066587, -0.34066587]])

In [66]:
train_data = rating_diff_new[:-1]
test_data = rating_diff_new[:-1]
query = rating_diff_new[-3]

In [67]:
user_similarity =  cosine_similarity(train_data, test_data).T

In [68]:
user_similarity

array([[ 1.        ,  0.10632193, -0.01642393, ...,  0.24748319,
         0.1155035 ,  0.10898367],
       [ 0.10632193,  1.        ,  0.08268017, ...,  0.19084806,
         0.14417664,  0.13829245],
       [-0.01642393,  0.08268017,  1.        , ...,  0.12753887,
         0.08453195,  0.10202478],
       ..., 
       [ 0.24748319,  0.19084806,  0.12753887, ...,  1.        ,
         0.12247753,  0.22123376],
       [ 0.1155035 ,  0.14417664,  0.08453195, ...,  0.12247753,
         1.        ,  0.08012962],
       [ 0.10898367,  0.13829245,  0.10202478, ...,  0.22123376,
         0.08012962,  1.        ]])

In [69]:
users = list(user_similarity[0])
sorted_users = sorted(users,reverse=True)
list_top_users  = sorted_users[:5]

In [91]:
pred = np.zeros(train[0].shape)
for i in list_top_users:
    index = users.index(i)
    pred += i*train[index]

In [92]:
sortpred = sorted(pred,reverse=True)
top_pred = sortpred[:15]
indexes = []
pred_list = list(pred)
for i in top_pred:
    if (pred_list.index(i) not in indexes):
        indexes.append(pred_list.index(i))

## Taking out recommendations for one user :  for example 5th user in the data

In [109]:
test_user = user_data_matrix[4]

In [110]:
test_user

array([ 4.,  3.,  0., ...,  0.,  0.,  0.])

In [111]:
movies_watched_index = []
for i in range(len(test_user)):
    if (test_user[i]>0):
        movies_watched_index.append(i+1)

In [112]:
movies_watched=[]
for i in movies_watched_index:
    movies_watched.append(movies['movie_title'][movies['movie_id'] == i])

In [113]:
movie_to_watch = []
for i in (indexes):
    if (i not in movies_watched_index):
        movie_to_watch.extend(movies_data['movie_title'][movies_data['movie_id'] == i+1])

In [114]:
if (len(movie_to_watch) > len(movies_watched)):
    movie_watched.extend(['']*(len(movie_to_watch) - len(movies_watched)))
else:
    movie_to_watch.extend(['']*(len(movies_watched) - len(movie_to_watch)))

In [115]:
recommend = pd.DataFrame({"Movies already rated": movies_watched,
                         "Movies recommended":movie_to_watch})

In [116]:
recommend

,Movies already rated,Movies recommended
0,"0 Toy Story (1995) Name: movie_title, dtype...",Star Wars (1977)
1,"1 GoldenEye (1995) Name: movie_title, dtype...",Jurassic Park (1993)
2,16 From Dusk Till Dawn (1996) Name: movie_t...,Toy Story (1995)
3,20 Muppet Treasure Island (1996) Name: movi...,Alien (1979)
4,23 Rumble in the Bronx (1995) Name: movie_t...,Dead Poets Society (1989)
5,"24 Birdcage, The (1996) Name: movie_title, ...",
6,"28 Batman Forever (1995) Name: movie_title,...",
7,"39 To Wong Foo, Thanks for Everything! Juli...",
8,"41 Clerks (1994) Name: movie_title, dtype: ...",
9,"49 Star Wars (1977) Name: movie_title, dtyp...",
